In [14]:
import pandas as pd
from config import (
    PATH_VP_POS,
    PATH_VP_ENT,
    PATH_TU_ENT,
    PATH_TU_UP,
    PATH_STOPS,
)

def load_raw_data():
    vp_pos = pd.read_csv(PATH_VP_POS)
    vp_ent = pd.read_csv(PATH_VP_ENT)
    tu_ent = pd.read_csv(PATH_TU_ENT)
    tu_up  = pd.read_csv(PATH_TU_UP)
    stops  = pd.read_csv(PATH_STOPS)
    return vp_pos, vp_ent, tu_ent, tu_up, stops

In [15]:
def load_raw_data():
    vp_pos = pd.read_csv(PATH_VP_POS)
    vp_ent = pd.read_csv(PATH_VP_ENT)
    tu_ent = pd.read_csv(PATH_TU_ENT)
    tu_up  = pd.read_csv(PATH_TU_UP)
    stops  = pd.read_csv(PATH_STOPS)
    return vp_pos, vp_ent, tu_ent, tu_up, stops

In [4]:
import pandas as pd
from config import (
    PATH_VP_POS,
    PATH_VP_ENT,
    PATH_TU_ENT,
    PATH_TU_UP,
    PATH_STOPS,
)

def load_raw_data():
    vp_pos = pd.read_csv(PATH_VP_POS)
    vp_ent = pd.read_csv(PATH_VP_ENT)
    tu_ent = pd.read_csv(PATH_TU_ENT)
    tu_up  = pd.read_csv(PATH_TU_UP)
    stops  = pd.read_csv(PATH_STOPS)
    return vp_pos, vp_ent, tu_ent, tu_up, stops

def load_raw_data():
    vp_pos = pd.read_csv(PATH_VP_POS)
    vp_ent = pd.read_csv(PATH_VP_ENT)
    tu_ent = pd.read_csv(PATH_TU_ENT)
    tu_up  = pd.read_csv(PATH_TU_UP)
    stops  = pd.read_csv(PATH_STOPS)
    return vp_pos, vp_ent, tu_ent, tu_up, stops

VEHICLES_COLUMNS = [
    "entity_id",
    "trip_id",
    "route_id",
    "vehicle_id",
    "latitude",
    "longitude",
    "current_stop_id",
    "current_stop_name",
    "current_stop_sequence",
    "current_status",
    "route_type",
    "timestamp",
]

TRIPS_COLUMNS = [
    "entity_id",
    "tu_trip_id",
    "tu_route_id",
    "tu_direction_id",
    "arrival_delay",
    "stop_id",
    "stop_sequence",
    "schedule_relationship",
    "route_type"
]

TRAM_ROUTES = {
    "1", "2", "3", "4", "5", "6", "7", "8", "9",
    "10A", "10B",
    "11",
    "12",
    "14",
    "15",
    "16", "17", "18", "19",
    "41", "43", "45",
}



BUS_ROUTES = {
    # dzienne / zwykłe / przyspieszone
    "Z", "Z3", "Z13",
    "50A", "50B",
    "51A", "51B",
    "52",
    "53A", "53B",
    "54A", "54B",
    "55A", "55B",
    "56",
    "57",
    "58A", "58B",
    "59",
    "60A", "60B", "60C", "60D",
    "61", "62", "63",
    "64A", "64B",
    "65A", "65B",
    "66",
    "68",
    "69A", "69B",
    "70",
    "71A", "71B",
    "72A", "72B",
    "73",
    "75A", "75B",
    "76",
    "77",
    "78",
    "80A", "80B",
    "81A", "81B",
    "82A", "82B",
    "83",
    "84A", "84B",
    "85A", "85B",
    "86",
    "87A", "87B",
    "88A", "88B", "88C", "88D",
    "89", "90",
    "91A", "91B", "91C",
    "92A", "92B",
    "93",
    "94",
    "96",
    "97A", "97B",
    "99",
    "201", "202",
    "F1", "G1", "G2", "H", "W",
}

NOCNE_BUS_ROUTES = {
    "N1A", "N1B",
    "N2",
    "N3A", "N3B",
    "N4A", "N4B",
    "N5A", "N5B",
    "N6",
    "N7A", "N7B",
    "N8",
    "N9",
}


def classify_route_type(route_id: str) -> str:
    """
    Zwraca 'tram', 'bus' lub 'unknown' na podstawie route_id.
    """
    if pd.isna(route_id):
        return "unknown"
    r = str(route_id).strip()

    if r in TRAM_ROUTES:
        return "tramwaj"
    if r in BUS_ROUTES:
        return "autobus"
    if r in NOCNE_BUS_ROUTES:
        return "nocny_autobus"
    return "unknown"
    

# 2. Dodanie nazw przystanków do vehicle_positions
def add_stop_names(vp_pos: pd.DataFrame, stops: pd.DataFrame) -> pd.DataFrame:
    """
    Dodaje nazwy przystanków do vehicle_positions na podstawie stops.txt
    Zakłada: current_stop_id = stop_id
    """
    stops_small = stops[["stop_id", "stop_name", "stop_lat", "stop_lon"]]

    merged = vp_pos.merge(
        stops_small,
        left_on="current_stop_id",
        right_on="stop_id",
        how="left"
    )

    merged = merged.rename(columns={
        "stop_name": "current_stop_name",
        "stop_lat": "current_stop_lat",
        "stop_lon": "current_stop_lon",
    }).drop(columns=["stop_id"])

    return merged


def build_vehicles_dataset(vp_pos: pd.DataFrame,
                           vp_ent: pd.DataFrame,
                           stops: pd.DataFrame) -> pd.DataFrame:

    vp_with_stops = add_stop_names(vp_pos, stops)

    vehicles = vp_with_stops.merge(
        vp_ent,
        on="entity_id",
        how="left",
        suffixes=("", "_vpent")
    )

    # 🔹 Dodajemy typ linii na podstawie route_id
    vehicles["route_type"] = vehicles["route_id"].apply(classify_route_type)

    # 🔹 Przycinamy do wybranych kolumn (dodaj route_type do VEHICLES_COLUMNS!)
    vehicles = vehicles[VEHICLES_COLUMNS].copy()

    return vehicles

def build_trips_dataset(tu_ent: pd.DataFrame,
                        tu_up: pd.DataFrame) -> pd.DataFrame:

    cols_keep = [
        "entity_id",
        "arrival_delay",
        "stop_id",
        "stop_sequence",
        "schedule_relationship",
    ]
    tu_up_small = tu_up[cols_keep]

    trips = tu_ent.merge(
        tu_up_small,
        on="entity_id",
        how="left",
        suffixes=("", "_update")
    )

    # 🔹 Dodajemy typ linii na podstawie tu_route_id
    trips["route_type"] = trips["tu_route_id"].apply(classify_route_type)

    # 🔹 Przycinamy do wybranych kolumn (dodaj route_type do TRIPS_COLUMNS!)
    trips = trips[TRIPS_COLUMNS].copy()

    return trips



# 5. Główny entrypoint
def build_datasets():
    vp_pos, vp_ent, tu_ent, tu_up, stops = load_raw_data()

    vehicles_df = build_vehicles_dataset(vp_pos, vp_ent, stops)
    trips_df    = build_trips_dataset(tu_ent, tu_up)

    return vehicles_df, trips_df

def build_datasets():
    """
    - wczytuje raw dane
    - buduje vehicles_df i trips_df
    - zapisuje oba do plików CSV
    - zwraca: vehicles_df, trips_df
    """
    vp_pos, vp_ent, tu_ent, tu_up, stops = load_raw_data()

    vehicles_df = build_vehicles_dataset(vp_pos, vp_ent, stops)
    trips_df    = build_trips_dataset(tu_ent, tu_up)

    # === ZAPIS CSV ===
    from config import OUTPUT_DIR

    vehicles_df.to_csv(OUTPUT_DIR / "vehicles_clean.csv", index=False, encoding="utf-8")
    trips_df.to_csv(OUTPUT_DIR / "trips_clean.csv", index=False, encoding="utf-8")

    print("✔ Saved:", OUTPUT_DIR / "vehicles_clean.csv")
    print("✔ Saved:", OUTPUT_DIR / "trips_clean.csv")

    return vehicles_df, trips_df

In [5]:
vehicles_df, trips_df = build_datasets()

✔ Saved: /Users/przemek/Documents/GitHub/Lodz_Hack/Wiki_Testing_Area/processed_output/vehicles_clean.csv
✔ Saved: /Users/przemek/Documents/GitHub/Lodz_Hack/Wiki_Testing_Area/processed_output/trips_clean.csv


In [46]:
trips_df.head()

,entity_id,tu_trip_id,tu_route_id,tu_direction_id,arrival_delay,stop_id,stop_sequence,schedule_relationship,route_type
0,trip_11312_3430,11312_3430,54B,0,-18,791,11,0,autobus
1,trip_11312_11025,11312_11025,Z3,1,-49,1375,13,0,autobus
2,trip_11312_5135,11312_5135,66,1,-21,1597,6,0,autobus
3,trip_11312_3382,11312_3382,54B,1,102,3757,9,0,autobus
4,trip_11312_3382,11312_3382,54B,1,102,3757,9,0,autobus


In [47]:
vehicles_df.head()

,entity_id,trip_id,route_id,vehicle_id,latitude,longitude,current_stop_id,current_stop_name,current_stop_sequence,current_status,route_type,timestamp
0,vehicle_1001,11312_3430,54B,1001,51.753361,19.511049,791,Niciarniana-Niciarniana 45 NŻ,11,0,autobus,1764759516
1,vehicle_1005,11312_11025,Z3,1005,51.811741,19.482809,1375,Wycieczkowa,13,2,autobus,1764759518
2,vehicle_1007,11312_5135,66,1007,51.838638,19.467390,1597,Łagiewnicka-Kuropatwia NŻ,6,0,autobus,1764759516
3,vehicle_1153,11312_3382,54B,1153,51.771992,19.576321,3757,Pomorska-Popielarnia NŻ,9,2,autobus,1764759510
4,vehicle_1158,11312_4810,64A,1158,51.796131,19.504499,98,Brzezińska-CH M1,12,2,autobus,1764759370


In [24]:
list(vehicles_df.columns)

['entity_id',
 'trip_id',
 'route_id',
 'vehicle_id',
 'latitude',
 'longitude',
 'speed',
 'bearing',
 'current_stop_id',
 'current_stop_sequence',
 'current_status',
 'timestamp',
 'congestion_level',
 'occupancy_status',
 'current_stop_name',
 'current_stop_lat',
 'current_stop_lon',
 'is_deleted',
 'has_trip_update',
 'has_vehicle_position',
 'has_alert',
 'vp_trip_id',
 'vp_route_id',
 'vp_vehicle_id',
 'vp_latitude',
 'vp_longitude',
 'vp_speed',
 'vp_bearing',
 'vp_current_stop_id',
 'vp_current_status',
 'vp_timestamp']

In [3]:
def build_datasets():
    """
    - wczytuje raw dane
    - buduje vehicles_df i trips_df
    - zapisuje oba do plików CSV
    - zwraca: vehicles_df, trips_df
    """
    vp_pos, vp_ent, tu_ent, tu_up, stops = load_raw_data()

    vehicles_df = build_vehicles_dataset(vp_pos, vp_ent, stops)
    trips_df    = build_trips_dataset(tu_ent, tu_up)

    # === ZAPIS CSV ===
    from config import OUTPUT_DIR

    vehicles_df.to_csv(OUTPUT_DIR / "vehicles_clean.csv", index=False, encoding="utf-8")
    trips_df.to_csv(OUTPUT_DIR / "trips_clean.csv", index=False, encoding="utf-8")

    print("✔ Saved:", OUTPUT_DIR / "vehicles_clean.csv")
    print("✔ Saved:", OUTPUT_DIR / "trips_clean.csv")

    return vehicles_df, trips_df

In [2]:
from config import OUTPUT_DIR, PROJECT_ROOT
from pathlib import Path
import os

print("PROJECT_ROOT:", PROJECT_ROOT)
print("OUTPUT_DIR  :", OUTPUT_DIR)
print("OUTPUT_DIR exists? ->", OUTPUT_DIR.exists())
print("Zawartość OUTPUT_DIR:")
if OUTPUT_DIR.exists():
    for p in OUTPUT_DIR.iterdir():
        print(" -", p.name)

PROJECT_ROOT: /Users/przemek/Documents/GitHub/Lodz_Hack
OUTPUT_DIR  : /Users/przemek/Documents/GitHub/Lodz_Hack/Wiki_Testing_Area/processed_output
OUTPUT_DIR exists? -> True
Zawartość OUTPUT_DIR:
